In [63]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import xavier_normal_
VOCAB_DIM = 300
MAX_LEN = 20
class Decoder(nn.Module):

    def __init__(self, dm, num_lay, emb):
        super(Decoder, self).__init__()
#         for construct cache positional encoding matrix.
        self.emb = emb
        self.rnn_cell = nn.GRUCell(dm, dm)
        self.linear = nn.Linear(dm, VOCAB_DIM+1)
    def stop(self, Q, hn):
        _, h = self.rnn(Q, hn)
        return h
        
        
    def decode(self, x):
        assert len(x.size()) == 2
        return torch.argmax(x, dim=-1)
    
    def forward(self, true_label, hn, teaching_rate):
        Q = true_label.clone()
        
        emb_out = self.emb(Q)
        batch, max_len = Q.size()
        output = []
        lengths = [1 for i in range(batch)]
        for i in range(max_len):
            if i == 0:
                Q = self.emb(true_label[:,0])
                print Q.shape, hn[-1,:,:].shape
                hx = self.rnn_cell(Q,hn[-1,:,:])
            else:
                if np.random.rand() < teaching_rate:
                    Q = self.emb(true_label[:,i])
                else:
                    pred_label = self.decode(hx)
                    
                    Q = self.emb(pred_label)
                    print 'Q',Q.shape
                hx = self.rnn_cell(Q,hx)
            output.append(hx)
                
        return output
class A(nn.Module):

    def __init__(self, dm, num_lay):
        super(A, self).__init__()
#         for construct cache positional encoding matrix.
        self.emb = nn.Embedding(VOCAB_DIM+1, dm, padding_idx=0)
        self.encoder = nn.GRU(dm, dm, num_lay, batch_first=True)
        self.linear = nn.Linear(dm, VOCAB_DIM+1)
        self.decoder = Decoder(dm, num_lay, self.emb)

    def forward(self, Q, K, input_lengths=None):
        batch, max_len = K.size()
        K = self.emb(K)
        if input_lengths:
            K = nn.utils.rnn.pack_padded_sequence(K, lengths, batch_first=True)
            
        en_out, hn = self.encoder(K) 
        if input_lengths:
            en_out, _ = nn.utils.rnn.pad_packed_sequence(en_out, batch_first=True, total_length=max_len)
#         y = self.linear(en_out)
        self.decoder(Q,hn,0.1)
#         return y
m = A(128,2)
bat = 7
# K = torch.randint(VOCAB_DIM+1,[bat,MAX_LEN])
Q = torch.arange(bat*MAX_LEN).view([bat, MAX_LEN])
K = torch.arange(bat*MAX_LEN).view([bat, MAX_LEN])
lengths = [9,8,6,6,4,3,2,]
o = m(Q, K, lengths)
# print K
# print o.shape

torch.Size([7, 128]) torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
Q torch.Size([7, 128])
